# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Импорт библиотек

In [1]:
import pandas as pd
from pymystem3 import Mystem

##  Получение данных

Откроем файл `data.csv` и сохраним в переменной `data`.

In [2]:
data = pd.read_csv('/datasets/data.csv')

Выведем первые 5 строк таблицы с помощью метода `head()`.

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Общую информацию о данных таблицы `data` получим с помощью метода `info()`.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<p>Исходя из общей информации о файле с данными, в таблице 12 колонок:</p>
<ol>
    <li>5 колонок с типом данных int64</li>
    <li>5 колонок с типом данных object</li>
    <li>2 колонки с типом данных float64</li>
</ol>
<p>В именованиях столбцов нарушений нет. Однако имеются пропуски в столбцах <b>days_employed</b> и <b>total_income</b>.
    Согласно документации к данным:
</p>
<ul>
    <li>
        <b>days_employed</b> - общий трудовой стаж в днях;
    </li>
    <li>
        <b>total_income</b> - ежемесячный доход;
    </li>
</ul>

**Вывод**

<p>В каждой строке таблице - данные о клиенте банка, подающего заявку на кредит. В рамках данного исследования, предварительно, можно сказать, что данных достаточно для проверки гипотез. Однако в данных имеются пропуски, нет общего стиля заполнения колонок, как, например, наличие отрицательных значений или различие регистра строковых значений, возможно наличие дубликатов.</p>
<p>Чтобы перейти к решению основных задач исследования, нужно устранить проблемы в данных.</p>

## Предобработка данных

В этой части мы:
1. Найдем и заполним пропущенные значения;
2. Приведем типы данных столбцов к целочисленному;
3. Проверим наличие дубликатов и удалим их, если таковые имеются;
4. Выполним лемматизацию;
5. Выполним категоризацию.

### Обработка пропусков

Проверим пропуски в данных. Воспользуемся методом `isna()` для поиска пропусков и методом `sum()` для получения их количества.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим, что в колонках `data_employed` и `total_income` 2174 пропуска, что составляет 10%. Величина пропусков довольно большая, поэтому необходимы корректировки в данных.

`data_employed` - трудовой стаж, в рамках исследования не рассматривается. Параметр учитывается при рассмотрении заявки на кредитный продукт и влияет на одобрение. В связи с этим от него можно отказаться полностью.
`total_income` - ежемесячный доход, оказывает влияние на возврат кредита в срок, именно поэтому пропуски этого параметра необходимо заполнить. В идеале заполнить, уточнив данные у тех, кто предоставляет статистику по клиентам. Так как такой возможности в рамках исследования мы не имеем, то можем лишь предположить природу их появления:
1. когда подается заявка, клиент прикладывает документы, подтверждающие доход. Возможно эти пропуски - те клиенты, которые не предоставили подтверждающих документов;
2. люди, занимающие руководящие должности и/или получившие образование в престижном вузе чаще, чем другие респонденты, не отвечают на вопрос о своих доходах.

Так как пропусков достоточно много, то будем рассматривать вариант того, что данные пропущены не случайно, а ввиду некоторых закономерностей. В связи с этим есть 2 варианта:
1. вероятность пропуска может быть определена на основе другой имеющейся в наборе данных информации (пол, возраст, занимаемая должность, образование…), не содержащей пропуски. В таком случае удаление или замена пропусков на значение «Пропуск», не приведет к существенному искажению результатов;
2. вероятность пропуска могла бы быть описана на основе других атрибутов, но информация по этим атрибутам в наборе данных отсутствует. Как следствие, вероятность пропуска невозможно выразить на основе информации, содержащейся в наборе данных. В таком случае просто игнорировать или исключить пропуски уже нельзя, так как это приведет к значительному искажению результатов исследования.

В связи с тем, что определить вероятность появления пропуска не представляется возможным в рамках данного исследования, то не будем игнорировать или исключать пропуски, чтобы как можно меньше избежать искажения. Для этого мы группируем по типу занятости и найдем среднее для каждого, а этими значениями заполним пропуски в соответствии с типом занятости.

Заменим старый столбец с пропусками в `total_income` на новый, с медианными значениями вместо пропусков.

In [6]:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

<p>Удалим стоблец <b>data_employed</b>:</p>

In [7]:
data.dropna(axis=1, inplace=True)

<p>Перепроверим на пропуски:</p>

In [8]:
data.isna().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**
<p>Таким образом, в данных были найдены пропуски в <b>data_employed</b> и <b>total_income</b>.</p>

<p>Пропуски в <b>total_income</b> были заменены на средние значения из доходов, соответствующих типу занятости.</p>

<p><b>data_employed</b> не используется для расчетов в рамках исследования, поэтому эта колонка с пропусками была удалена.</p>

### Замена типа данных

В этом блоке нас интересует приведение вещественного типа данных к целочисленному. Поэтому еще раз подробно рассмотрим их с помощью атрибута `dtypes`.

In [9]:
data.dtypes

children              int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Видим, что вещественный тип встречается только в столбце <b>total_income</b>.
Воспользуемся методом <code>astype()</code>.

In [10]:
data['total_income'] = data['total_income'].astype('int64')

In [11]:
data.dtypes

children             int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Значения в столбце `total_income` переведены в целочисленный тип данных. Для изменения пользовались методом <code>astype()</code> потому, что метод <code>to_numeric()</code> приводит к типу **float**, а нам нужно привести к **int**.

### Обработка дубликатов

Методы <code>duplicated()</code> и <code>drop_duplicates()</code> помогают избавить от полных дубликатов. Для этого нужно проверить и подготовить данные во всех столбцах. Пройдем по порядку:

1. <b>children</b>

In [12]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

а) Видим, что среди значений есть отрицательное, чего не может быть. Скорее всего вместо -1 имелось ввиду 1;<br>
б) Так же видим, что есть значение 20, что, в принципе, возможно, но маловероятно. Поэтому заменим 20 на 2;

In [13]:
def correct_children_numbers(number):
    if number == 20: # блок if будет отвечать за изменение значения 20 на 2
        return int(number / 10)
    return max(-number, number) # с помощью max вернем положительное число вместо отрицательного

In [14]:
data['children'] = data['children'].apply(correct_children_numbers)

In [15]:
# проверка
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

2. <b>dob_years</b>

In [16]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Видим, что среди уникальных значений присутствует возраст 0 лет, чего не может быть. Проверим, сколько заемщиков с некорректным возрастом по типу занятости.

In [17]:
data[data['dob_years'] == 0]['income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

101 заемщик с возрастом 0 лет - непорядок. Заменим нулевые значения на средние в соответствии с типом занятости.

In [18]:
data_grouped_years = data.groupby('income_type')['dob_years'].mean()
data_grouped_years

income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.636737
компаньон          39.697542
пенсионер          59.063019
предприниматель    42.500000
сотрудник          39.821027
студент            22.000000
Name: dob_years, dtype: float64

In [19]:
for i in data['income_type'].unique():
    mean_years = data.loc[data['income_type'] == i, 'dob_years'].mean()
    data.loc[(data['dob_years'] == 0), 'dob_years'] = mean_years

In [20]:
# проверка
data[data['dob_years'] == 0]['income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

In [21]:
data.dtypes

children              int64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

Видим, что нулевого возраста в данных больше нет. Однако мы заменяли на среднее значение, что привело изменению типа данных столбца <b>dob_years</b> на вещественный. Приведем к числовому по аналогии с пунктом 2.2

In [22]:
data['dob_years'] = data['dob_years'].astype('int64')

In [23]:
data.dtypes

children             int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

3. <b>education</b>

In [24]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Видим, что при заполнении информации об образовании не учитывался регистр. Нужно привести к нижнему.

In [25]:
data['education'] = data['education'].str.lower()

In [26]:
# проверка
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

4. <b>education_id</b>

In [27]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

После внесения правок в <b>education</b> количество уникальных значений совпадает с <b>education_id</b>. Тут все в порядке. 

5. <b>family_status</b> и 6. <b>family_status_id</b>

In [28]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Сверим с <b>family_status_id</b>

In [29]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Совпадают. Все в порядке, двигаемся дальше.

7. <b>gender</b>

In [30]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [31]:
data[data['gender'] == 'XNA']

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Заемщик с гендером XNA - единственный в данных. Пусть это будет еще один тип гендера, отличный от основных. Исследованию не помешает.

8. <b>debt</b>

In [32]:
data['debt'].unique()

array([0, 1])

Тут все ок. Если задолженность есть, то 1. Если же её нет, то 0.

9. <b>total_income</b><br>
Уникальных значений дохода будет очень много. Воспользуемся методом <code>value_counts()</code>

In [33]:
data['total_income'].value_counts()

142594    1105
172357     509
118514     414
150447     147
126262       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: total_income, Length: 18608, dtype: int64

10. <b>purpose</b>

In [34]:
words_purpose = data['purpose'].unique()
words_purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Видим, что одни и те же по смыслу цели записаны по-разному. Нужно одинаковые привести к одному. Нужна лемматизация. Для этого необходимо подключить библиотеку <b>pymystem3</b>. Займемся лемматизацией в следующем блоке.

Теперь, когда все столбцы данных просмотрены, можно искать дубликаты. 

In [35]:
data.duplicated().sum()

71

Видим, что в данных имеются дубликаты, общая сумма которых равна 71. Удалим дубликаты с помощью метода <code>drop_duplicates()</code>.

In [36]:
# добавим к результату метод reset_index() со значением drop = True, чтобы не создавать столбец со старыми значениями индексов.
data = data.drop_duplicates().reset_index(drop = True)

In [37]:
# проверка
data.duplicated().sum()

0

**Вывод**

Мы внесли корректировки в данных, поправили строковые и числовые значения и по итогу нашли дубликаты в количестве 71, от которых благополучно избавились. Эта часть данных подготовлена. Можем переходить к лемматизации.

### Лемматизация

Получим лемматизатор для слов на русском языке из библиотеки `pymystem3`.

In [38]:
m = Mystem()

Определим базовые леммы в строке, полученной из списка уникальных значений столбца <b>purpose</b>

In [39]:
lemmas = m.lemmatize(' '.join(words_purpose))
lemmas

['покупка',
 ' ',
 'жилье',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',
 'дополнительный',
 ' ',
 'образование',
 ' ',
 'сыграть',
 ' ',
 'свадьба',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ' ',
 'образование',
 ' ',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ' ',
 'покупка',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'недвижимость',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'жилье',
 ' ',
 'операция',
 ' ',
 'со',
 ' ',
 'свой',
 ' ',
 'недвижимо

Среди лемм видим, что самые общие это:
<ol>
    <li>автомобиль</li>
    <li>образование</li>
    <li>недвижимость (куда можно отнести и "жилье")</li>
    <li>свадьба</li>
</ol>

In [40]:
# создание списка общих лемм
main_purpose_list = ['автомобиль', 'образование', 'недвижимость', 'свадьба']

Напишем функцию, которая проверит вхождение информации в список общих лемм. В случае вхождения вернет общую.

In [41]:
def change_purpose(purpose):
    # проходим по списку общих лемм
    for purp in main_purpose_list:
        # проверка есть ли общая лемма в строке из лемм, полученных с помощью лемматизатора из значения purpose.
        # в этой же строке сразу меняем "жилье" на "недвижимость"
        if purp in ' '.join(m.lemmatize(purpose)).replace('жилье', 'недвижимость'):
            # если да, то вернем эту общую лемму
            return purp

Воспользуемся функцией `change_purpose()` и произведем замену.

In [42]:
data['main_purpose'] = data['purpose'].apply(change_purpose)

In [43]:
# проверка
data['main_purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

**Вывод**

Таким образом, разные вариации написания одной и той же цели мы привели к общей. Теперь у нас всего 4 категории цели для получения кредита.

### Категоризация данных

Еще раз посмотрим на данные. Выведем первые 10 строк.

In [44]:
data.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,main_purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


Для ответов на вопросы исследования нам нужно знать:<br>
<ol>
    <li>есть ли дети</li>
    <li>семейное положение</li>
    <li>уровень дохода</li>
    <li>цель кредита</li>
    <li>задолженность</li>
</ol>
Поэтому для упрощения нужно категоризировать все эти данные.

1. <b>Дети</b>. Если детей нет - категория "без детей". Если есть - "с детьми".

In [45]:
def categorize_children_count(num):
    if num == 0:
        return 'без детей'
    return 'с детьми'

In [46]:
data['children_id'] = data['children'].apply(categorize_children_count)

In [47]:
# проверка
data['children_id'].unique()

array(['с детьми', 'без детей'], dtype=object)

2. <b>Семейное положение</b>. В данных есть <b>family_status_id</b>, соответствующий данным в <b>family_status</b>

3. <b>Уровень дохода</b>. Разделим по такому принципу:
<ul>
    <li>до 50 000 - крайне низкий</li>
    <li>от 50 до 100 - низкий</li>
    <li>от 100 до 200 - нормальный</li>
    <li>от 200 до 500 - выше среднего</li>
    <li>от 500 - высокий</li>
</ul>

In [48]:
def categorize_total_income(income):
    
    if income <= 50000:
        return 'крайне низкий'
    elif (income > 50000) & (income <= 100000):
        return 'низкий'
    elif (income > 100000) & (income <= 200000):
        return 'нормальный'
    elif (income > 200000) & (income <= 500000):
        return 'выше среднего'
    else:
        return 'высокий'

In [49]:
data['total_income_id'] = data['total_income'].apply(categorize_total_income)

In [50]:
# проверка
data['total_income_id'].unique()

array(['выше среднего', 'нормальный', 'низкий', 'высокий',
       'крайне низкий'], dtype=object)

4. <b>Цель кредита</b>. У нас уже есть разделение по основным категориям из блока лемматизации.

5. <b>Задолженность</b>. Эта графа уже представлено в виде категорий, где 0 - нет задолженности, а 1 - есть задолженность.

**Вывод**

Таким образом, мы выполнили категоризацию данных, необходимых для исследования. Можем переходить к проверкам.

## Проверка зависимостей

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим новый датафрейм для представления данных по этой зависимости:

In [51]:
debt_child_relation = pd.DataFrame()

Добавим в новый фрейм количество заемщиков, сгруппированных по наличию/отсутствию детей:

In [52]:
debt_child_relation['borrows_count'] = data.groupby('children_id')['debt'].count()

Просуммируем значения в этой же группе - получим количество должников. Так же добавим в новый фрейм:

In [53]:
debt_child_relation['borrows_sum'] = data.groupby('children_id')['debt'].sum()

Найдем отношение суммарного значения к количественному и так же добавим для наглядности:

In [54]:
debt_child_relation['result'] = (debt_child_relation['borrows_sum'] / debt_child_relation['borrows_count']) * 100

Отсортируем по убыванию и выведем на экран:

In [55]:
debt_child_relation.sort_values('result', ascending = False)

,borrows_count,borrows_sum,result
children_id,,,
с детьми,7363,678,9.208203
без детей,14091,1063,7.543822


**Вывод**

Как видно из таблицы, результат не сильно отличается - есть ли у заемщика дети или нет. Получается, что зависимость между наличием детей и возвратом кредита в срок есть, но она не большая - разница примерно 1.7%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Аналогично проведем исследование зависимости между семейным положением и возвратом кредита в срок:

In [56]:
debt_family_relation = pd.DataFrame()

In [57]:
debt_family_relation['debt_count'] = data.groupby('family_status')['debt'].count()
debt_family_relation['debt_sum'] = data.groupby('family_status')['debt'].sum()
debt_family_relation['debt_result'] = (debt_family_relation['debt_sum'] / debt_family_relation['debt_count']) * 100

In [58]:
debt_family_relation.sort_values('debt_result', ascending = False)

,debt_count,debt_sum,debt_result
family_status,,,
Не женат / не замужем,2810,274,9.750890
гражданский брак,4151,388,9.347145
женат / замужем,12339,931,7.545182
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343


**Вывод**

По результатам видим, что зависимость так же, как и в предыдущем исследовании - небольшая. Максимальная разница составляет 2.6%. При этом минимальный риск у овдовевших людей, а максимальный - у не женатых/не замужних.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Таким же образом посчитаем зависимость между уровнем дохода и возвратом в срок.

In [59]:
debt_income_relation = pd.DataFrame()

In [60]:
debt_income_relation['debt_count'] = data.groupby('total_income_id')['debt'].count()
debt_income_relation['debt_sum'] = data.groupby('total_income_id')['debt'].sum()
debt_income_relation['debt_result'] = (debt_income_relation['debt_sum'] / debt_income_relation['debt_count']) * 100

In [61]:
debt_income_relation.sort_values('debt_result', ascending = False)

,debt_count,debt_sum,debt_result
total_income_id,,,
нормальный,11924,1029,8.629654
низкий,4091,331,8.090931
выше среднего,4845,344,7.100103
высокий,222,14,6.306306
крайне низкий,372,23,6.182796


**Вывод**

Из результатов видно, что разница так же составляет около 2%. Крайне низкий уровень дохода менеше всего влияет на возврат в срок. Скорее всего это обусловлено тем, что люди с таким доходом гораздо реже берут кредиты.

Более высокое влияние на возврат у заемщиков с нормальным уровне дохода - их много.

- Как разные цели кредита влияют на его возврат в срок?

In [62]:
debt_purpose_relation = pd.DataFrame()

In [63]:
debt_purpose_relation['debt_count'] = data.groupby('main_purpose')['debt'].count()
debt_purpose_relation['debt_sum'] = data.groupby('main_purpose')['debt'].sum()
debt_purpose_relation['debt_result'] = (debt_purpose_relation['debt_sum'] / debt_purpose_relation['debt_count']) * 100

In [64]:
debt_purpose_relation.sort_values('debt_result', ascending = False)

,debt_count,debt_sum,debt_result
main_purpose,,,
автомобиль,4306,403,9.359034
образование,4013,370,9.220035
свадьба,2324,186,8.003442
недвижимость,10811,782,7.233373


**Вывод**

Похожая картина и в этом исследовании. Минимальное влияние оказывает кредит на недвижимость. Скорее всего это обусловлено тем, что, чаще всего это ипотека и заемщики готовятся к платежу заранее, чтобы не остаться без дома. 

Максимальное влияние - кредит на автомобиль. Возможно, он не всегда оправдывает себя, как покупка в кредит, поэтому заемщики иногда пренебрегают платой в срок.

## Общий вывод

Мы провели 4 исследования.<br>
По итогу можно сказать, что все 4 исследования подтверждают наличия влияния на возврат кредита в срок. Причем некоторые категории на немного больше оказывают влияние на возврат кредита.

Данные могли немного исказиться по причине удаления пропусков и отказа от данных о трудовом стаже.

В целом на возврат кредита одновременно оказывают влияние много факторов, большая часть которых, уверен, не освящается в приведенной статистике. Поэтому для более глубоко исследования необходимы дополнительные данные.

## Чек-лист готовности проекта

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.